In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

csv_file_path='top200_training.xls'
#記得pip install xlrd
df = pd.read_excel(csv_file_path)
#df.head()

In [15]:
from itertools import product
from itertools import combinations


# 將年月轉換為日期型數據
df['年月'] = pd.to_datetime(df['年月'], format='%Y%m')

# 獲取所有不同的年份
all_years = ['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008']

# 開始的欄位
start_column = '1997'

# 開始欄位的索引
start_index = all_years.index(start_column)

# 創建一個字典來保存結果
year_combinations_dict = {}

# 生成所有可能的欄位組合
for i in range(1, len(all_years) - start_index + 1):
    for subset in combinations(all_years[start_index:], i):
        selected_columns = list(subset)
        # 將結果存入字典
        year_combinations_dict[len(year_combinations_dict) + 1] = selected_columns

# 打開一個檔案來保存結果
with open('year_combinations.txt', 'w') as file:
    # 遍歷字典，將結果寫入檔案
    for key, value in year_combinations_dict.items():
        file.write(f"Key: {key}, Selected year: {value}\n")
        # 在這裡進行你的分析或模型擬合等操作
        # ...
        file.write("\n" + "="*40 + "\n")

# 提示保存成功
print("Results saved to 'year_combinations.txt'")

Results saved to 'year_combinations.txt'


In [16]:
from itertools import combinations
from sklearn.model_selection import train_test_split
# 所有的欄位
all_columns = ['股價淨值比', '股價營收比' ,'M淨值報酬率─稅後', 'M營業利益成長率', 'M稅後淨利成長率']

# 開始的欄位
start_column = '股價淨值比'

# 開始欄位的索引
start_index = all_columns.index(start_column)

# 創建一個字典來保存結果
column_combinations_dict = {}

# 生成所有可能的欄位組合
for i in range(1, len(all_columns) - start_index + 1):
    for subset in combinations(all_columns[start_index:], i):
        selected_columns = list(subset)
        # 將結果存入字典
        column_combinations_dict[len(column_combinations_dict) + 1] = selected_columns

# 打開一個檔案來保存結果
with open('column_combinations_KNN_train.txt', 'w') as file:
    # 遍歷字典，將結果寫入檔案
    for key, value in column_combinations_dict.items():
        file.write(f"Key: {key}, Selected Columns: {value}\n")
        # 在這裡進行你的分析或模型擬合等操作
        # ...
        file.write("\n" + "="*40 + "\n")

# 提示保存成功
print("Results saved to 'column_combinations_KNN_train.txt'")


Results saved to 'column_combinations_KNN_train.txt'


In [ ]:
max_ret=0
ans={}
list_ans=[]
#選擇特定欄位分析
from sklearn.preprocessing import StandardScaler
from itertools import islice
cols=0
start_index = 1  # 第3001筆資料的索引
end_index = 4093  # 最後一筆資料的索引

selected_range = islice(year_combinations_dict.items(), start_index, end_index + 1)
#selected_range = islice(year_combinations_dict.items(), 4094) #4094是所有可用來訓練的年份組合

#16384
with open('KNN_train_test_result.txt', 'a') as file:
    for key, selected_years in selected_range:
        
        # 選擇訓練集年分
        train_years = [int(year) for year in selected_years]
        #print(train_years)

        #print("---------------------")
        # 選擇測試集年分
        test_years = [int(year) for year in all_years if year not in selected_years]
        print(f'總共有test year:{len(test_years)}')
        
        # 根據年份選擇訓練集
        train_data = df[df['年月'].dt.year.isin(train_years)]
        print(f"Number of rows in train_data: {len(train_data)}")
        #print(train_data)
        
        # 排除指定列，並標準化
        #columns_to_exclude = ['簡稱', '證券代碼', '年月', 'ReturnMean_year_Label', 'Return']
        #features_to_scale_train = train_data.drop(columns=columns_to_exclude)
        #features_to_scale_test = test_data.drop(columns=columns_to_exclude)
    
        for col in range (1,32):#取top10組合的前幾大frquency的屬性組合測試
            cols=cols+1
            list_ans=[]
            # 指定要選擇的欄位
            selected_columns = column_combinations_dict[col]
            list_ans.append(selected_columns)
            
            # 選擇指定的欄位
            features_to_scale_train = train_data[selected_columns]

            scaler = StandardScaler()
            scaled_features_train = scaler.fit_transform(features_to_scale_train)
            
            # 轉換為dataframe
            df_feat_train = pd.DataFrame(scaled_features_train, columns=features_to_scale_train.columns)
            
        #---------------------------------------------------------------------------------------------
            # 分割訓練集和測試集
            X_train = df_feat_train
            y_train = train_data['ReturnMean_year_Label']
            
            X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size=0.3,random_state=101)
  
        #--------------------------------------------------------------------------------------------------
            error_rate = []
            from sklearn.neighbors import KNeighborsClassifier
            for i in range(1, 100):
                knn = KNeighborsClassifier(n_neighbors=i, p=2, weights='distance', algorithm='brute')
                knn.fit(X_train, y_train)
                pred_i = knn.predict(X_test)
                error_rate.append(np.mean(pred_i != y_test))

            # 將k=1~60的錯誤率製圖畫出。k=23之後，錯誤率就在5-6%之間震盪。
            #plt.figure(figsize=(10, 6))
            #plt.plot(range(1, 100), error_rate, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10)
            #plt.title('Error Rate vs. K Value')
            #plt.xlabel('K')
            #plt.ylabel('Error Rate')
            #plt.show()
        #--------------------------------------------------------------------------------------------------
            min_error = min(error_rate)
            optimal_k = error_rate.index(min_error) + 1  # Adding 1 because Python indexing starts from 0

            print(f"Optimal k value for minimum error rate: {optimal_k}")
            #print(f"Minimum Error Rate: {min_error}")
            list_ans.append(optimal_k)
        #--------------------------------------------------------------------------------------------------
            #使用KNN演算法
            clf=KNeighborsClassifier(n_neighbors=optimal_k,p=2,weights='distance',algorithm='brute')
            clf.fit(X_train,y_train)
        #--------------------------------------------------------------------------------------------------
            predict_label1={}
            predict_k_pre5={}
        #--------------------------------------------------------------------------------------------------

            for year in test_years:
                
                # 選擇特定年份的 test_data
                test_data = df[df['年月'].dt.year == year]
                print(year)
                #print(test_data)
                print(f"Number of rows in test_data: {len(test_data)}")
                
                # 選擇指定的欄位
                features_to_scale_test = test_data[selected_columns]
                # 使用之前訓練好的標準化物件進行標準化
                scaled_features_test = scaler.transform(features_to_scale_test)
                # 將標準化後的特徵資料轉換為 DataFrame
                df_feat_test = pd.DataFrame(scaled_features_test, columns=features_to_scale_test.columns)

                
                # 使用已經訓練好的模型進行預測
                predictions_new = clf.predict(df_feat_test)

                #------------------------------
                #測試KNN演算法的好壞
                from sklearn.metrics import classification_report,confusion_matrix
                #將實際類別分為真正例（True Positive）、真負例（True Negative）、偽正例（False Positive）和偽負例（False Negative）
                print('confusion_matrix:')
                print(confusion_matrix(test_data['ReturnMean_year_Label'],predictions_new))

                #模型的精確度、召回率、F1分數和支持數等指標，用來評估模型對於每個類別的預測性能。
                print('classification_report')
                print(classification_report(test_data['ReturnMean_year_Label'],predictions_new))

                # 比較預測結果
                accuracy_new = clf.score(df_feat_test, test_data['ReturnMean_year_Label'])
                print(f'新數據的預測準確率: {accuracy_new}')
                #---------------------------------------
                #print(predictions_new)

                #predicted_positive_indices = (predictions_new == 1)

                # 獲取股票名稱
                #predicted_positive_stock_names = new.loc[predicted_positive_indices, '簡稱']

                # 設定檔案名稱
                #output_file_name = 'selected_stocks_1998.csv'

                # 匯出成 CSV 檔案
                #predicted_positive_stock_names.to_csv(output_file_name, index=True)

                # 預測要投資的股票名稱
                #print("選擇股票:")
                #print(predicted_positive_stock_names)


                # 選擇預測為1的股票
                selected_stocks = test_data[predictions_new == 1]

                # 計算return
                stock_returns = selected_stocks['Return']

                portfolio_returns = (stock_returns.mean()/100)+1

                print(portfolio_returns)
                predict_label1[year]=portfolio_returns

                # 使用 kneighbors 方法取得最近鄰居的索引和距離
                distances, indices = clf.kneighbors(df_feat_test, n_neighbors=5)

                # 合併所有測試樣本的最近鄰居索引
                all_indices = np.concatenate(indices)
                all_distances = np.concatenate(distances)

                # 將索引和距離組合成一個 2D 陣列，方便排序
                combined_data = np.column_stack((all_indices, all_distances))

                # 按照距離重新排序
                sorted_combined_data = combined_data[np.argsort(combined_data[:, 1])]

                # 選取最近鄰居中預測值為1的前5筆，且不重複
                selected_indices = set()
                i = 0
                while len(selected_indices) < 5 and i < len(sorted_combined_data):
                    index = int(sorted_combined_data[i, 0])
                    if(index>=200 and index<400):
                        index=index-200
                    elif(index>=400 and index<600):
                        index=index-400
                    elif(index>=600 and index<800):
                        index=index-600
                    elif(index>=800 and index<1000):
                        index=index-800
                    elif(index>=1000 and index<1200):
                        index=index-1000
                    elif(index>=1200 and index<1400):
                        index=index-1200
                    elif(index>=1400 and index<1600):
                        index=index-1400
                    elif(index>=1600 and index<1800):
                        index=index-1600
                    elif(index>=1800 and index<2000):
                        index=index-1800
                    elif(index>=2000 and index<2200):
                        index=index-2000
                    elif(index>=2200 and index<2400):
                        index=index-2200
                    elif(index>=2400 and index<2600):
                        index=index-2400
                    elif(index>=2600 and index<2800):
                        index=index-2600
                    elif(index>=2800 and index<3000):
                        index=index-2800
                    elif(index>=3000 and index<3200):
                        index=index-3000
                    prediction = predictions_new[index]
                    if prediction == 1 and index not in selected_indices:
                        selected_indices.add(index)
                    i += 1

                # 將選取的索引轉換成列表
                selected_indices_list = list(selected_indices)

                # 打印最近鄰居中預測值為1的前5筆索引
                print("最近鄰居中預測值為1的前5筆索引:")
                print(selected_indices_list)
                select = test_data
                # 獲取股票名稱
                selects_stock = select.iloc[selected_indices_list]['簡稱']
                print(selects_stock)
                # 獲取股票return
                selects_ret = select.iloc[selected_indices_list]['Return']
                print(selects_ret)
                with open(f'selected_stock_KNN_train/{cols}.txt', 'a') as stockfile:
                    stockfile.write(selects_stock.to_string())
                    stockfile.write('\n')
                    stockfile.write(selects_ret.to_string())
                # 計算return
                stock_returns = (selects_ret.mean() / 100)+1
                portfolio_returns = stock_returns
                print(portfolio_returns)
                predict_k_pre5[year] = portfolio_returns
                #print(stock_returns)
            sum1=1
            sum2=1
            print(predict_label1)
            for k in predict_label1.keys():
                if not (predict_label1[k]==0):
                    sum1=predict_label1[k]*sum1
            print(f'年均化複利為:{sum1/int(len(test_years))}')
            print('\n')
            print(predict_k_pre5)
            for k in predict_k_pre5.keys():
                if not (predict_k_pre5[k]==0):
                    sum2=predict_k_pre5[k]*sum2
            print(f'年均化複利為:{sum2/int(len(test_years))}')
            #list_ans.append(sum2/len(test_data)/200)
            print(f'ori sum2:{sum2}')
            print(f'test years have:{int(len(test_years))}')
            sum2=sum2/int(len(test_years))
            print(f'after sum2:{sum2}')
            list_ans.append(sum2)
            ans[cols]=list_ans
            file.write(f'{cols}:{list_ans}\n\n')
            file.flush()  # 強制將緩衝區內容寫入檔案
            if(sum2>max_ret):
                max_ret=sum2
                with open('max_KNN_train.txt', 'w') as maxfile:
                    maxfile.write(f'max:train_year:{train_years},test_year:{test_years}:{cols}:{list_ans}\n\n')
                    maxfile.flush()  # 強制將緩衝區內容寫入檔案

總共有test year:11
Number of rows in train_data: 200
Optimal k value for minimum error rate: 67
1998
Number of rows in test_data: 200
confusion_matrix:
[[69 57]
 [40 34]]
classification_report
              precision    recall  f1-score   support

          -1       0.63      0.55      0.59       126
           1       0.37      0.46      0.41        74

    accuracy                           0.52       200
   macro avg       0.50      0.50      0.50       200
weighted avg       0.54      0.52      0.52       200

新數據的預測準確率: 0.515
1.1148766703296702
最近鄰居中預測值為1的前5筆索引:
[137, 74, 13, 46, 84]
337    云辰         
274    中工         
213    統一         
246     中石化       
284    永大         
Name: 簡稱, dtype: object
337    58.8567
274   -37.7753
213    -0.8643
246   -13.4566
284   -39.3594
Name: Return, dtype: float64
0.9348022
1999
Number of rows in test_data: 200
confusion_matrix:
[[77 34]
 [47 42]]
classification_report
              precision    recall  f1-score   support

          -1       0.6

2.5603743999999997
{1998: 1.1148766703296702, 1999: 0.6161642236842106, 2000: 1.3592645289256198, 2001: 1.0614596666666667, 2002: 1.3691212201010101, 2003: 1.0875387620689656, 2004: 1.1147295465753424, 2005: 1.404274577142857, 2006: 1.1688538295081967, 2007: 0.5427639230769231, 2008: 2.105020606382978}
年均化複利為:0.2804618679528036


{1998: 0.9348022, 1999: 0.4402458, 2000: 0.9475709999999999, 2001: 0.9146118, 2002: 1.1204276, 2003: 0.94518866, 2004: 0.9563232, 2005: 1.5150481999999998, 2006: 1.1342576, 2007: 0.4794286, 2008: 2.5603743999999997}
年均化複利為:0.06926971132156413
ori sum2:0.7619668245372055
test years have:11
after sum2:0.06926971132156413
Optimal k value for minimum error rate: 9
1998
Number of rows in test_data: 200
confusion_matrix:
[[67 59]
 [31 43]]
classification_report
              precision    recall  f1-score   support

          -1       0.68      0.53      0.60       126
           1       0.42      0.58      0.49        74

    accuracy                           0.55 

Optimal k value for minimum error rate: 7
1998
Number of rows in test_data: 200
confusion_matrix:
[[88 38]
 [36 38]]
classification_report
              precision    recall  f1-score   support

          -1       0.71      0.70      0.70       126
           1       0.50      0.51      0.51        74

    accuracy                           0.63       200
   macro avg       0.60      0.61      0.61       200
weighted avg       0.63      0.63      0.63       200

新數據的預測準確率: 0.63
1.2076406315789474
最近鄰居中預測值為1的前5筆索引:
[97, 3, 70, 84, 21]
297    國聯         
203    中鋼         
270    大陸         
284    永大         
221    華宇         
Name: 簡稱, dtype: object
297   -21.2843
203    39.6414
270   -21.0983
284   -39.3594
221    23.9331
Name: Return, dtype: float64
0.963665
1999
Number of rows in test_data: 200
confusion_matrix:
[[74 37]
 [46 43]]
classification_report
              precision    recall  f1-score   support

          -1       0.62      0.67      0.64       111
           1       0.54

Optimal k value for minimum error rate: 21
1998
Number of rows in test_data: 200
confusion_matrix:
[[81 45]
 [52 22]]
classification_report
              precision    recall  f1-score   support

          -1       0.61      0.64      0.63       126
           1       0.33      0.30      0.31        74

    accuracy                           0.52       200
   macro avg       0.47      0.47      0.47       200
weighted avg       0.51      0.52      0.51       200

新數據的預測準確率: 0.515
1.032706223880597
最近鄰居中預測值為1的前5筆索引:
[101, 71, 105, 47, 95]
301    大榮         
271    燿華         
305    巨大         
247    台鳳         
295    長億         
Name: 簡稱, dtype: object
301   -52.4020
271   -24.7732
305   -41.9580
247   -54.5685
295   -33.8082
Name: Return, dtype: float64
0.5849802
1999
Number of rows in test_data: 200
confusion_matrix:
[[76 35]
 [58 31]]
classification_report
              precision    recall  f1-score   support

          -1       0.57      0.68      0.62       111
           1      

Optimal k value for minimum error rate: 45
1998
Number of rows in test_data: 200
confusion_matrix:
[[85 41]
 [47 27]]
classification_report
              precision    recall  f1-score   support

          -1       0.64      0.67      0.66       126
           1       0.40      0.36      0.38        74

    accuracy                           0.56       200
   macro avg       0.52      0.52      0.52       200
weighted avg       0.55      0.56      0.56       200

新數據的預測準確率: 0.56
1.141477411764706
最近鄰居中預測值為1的前5筆索引:
[0, 99, 35, 10, 13]
200     台積電       
299     三商行       
235    大眾         
210    華通         
213    統一         
Name: 簡稱, dtype: object
200    193.2025
299     10.7232
235      5.6047
210      5.8307
213     -0.8643
Name: Return, dtype: float64
1.4289936
1999
Number of rows in test_data: 200
confusion_matrix:
[[76 35]
 [57 32]]
classification_report
              precision    recall  f1-score   support

          -1       0.57      0.68      0.62       111
           1     

...

              precision    recall  f1-score   support

          -1       0.67      0.57      0.62       126
           1       0.41      0.51      0.46        74

    accuracy                           0.55       200
   macro avg       0.54      0.54      0.54       200
weighted avg       0.57      0.55      0.56       200

新數據的預測準確率: 0.55
1.203771097826087
最近鄰居中預測值為1的前5筆索引:
[5, 134, 71, 47, 94]
205    鴻海         
334    正隆         
271    燿華         
247    台鳳         
294    國豐         
Name: 簡稱, dtype: object
205    87.7049
334     4.6178
271   -24.7732
247   -54.5685
294   -64.0000
Name: Return, dtype: float64
0.8979619999999999
2000
Number of rows in test_data: 200
confusion_matrix:
[[58 69]
 [36 37]]
classification_report
              precision    recall  f1-score   support

          -1       0.62      0.46      0.52       127
           1       0.35      0.51      0.41        73

    accuracy                           0.48       200
   macro avg       0.48      0.48      0.47

confusion_matrix:
[[70 56]
 [40 34]]
classification_report
              precision    recall  f1-score   support

          -1       0.64      0.56      0.59       126
           1       0.38      0.46      0.41        74

    accuracy                           0.52       200
   macro avg       0.51      0.51      0.50       200
weighted avg       0.54      0.52      0.53       200

新數據的預測準確率: 0.52
1.1088586777777778
最近鄰居中預測值為1的前5筆索引:
[130, 6, 135, 51, 93]
330    永記         
206     英業達       
335    敬鵬         
251    矽統         
293    台苯         
Name: 簡稱, dtype: object
330    -25.5993
206    -13.1390
335     -2.5884
251    154.6171
293     49.2027
Name: Return, dtype: float64
1.3249862000000001
2000
Number of rows in test_data: 200
confusion_matrix:
[[61 66]
 [40 33]]
classification_report
              precision    recall  f1-score   support

          -1       0.60      0.48      0.54       127
           1       0.33      0.45      0.38        73

    accuracy                   

confusion_matrix:
[[72 54]
 [42 32]]
classification_report
              precision    recall  f1-score   support

          -1       0.63      0.57      0.60       126
           1       0.37      0.43      0.40        74

    accuracy                           0.52       200
   macro avg       0.50      0.50      0.50       200
weighted avg       0.54      0.52      0.53       200

新數據的預測準確率: 0.52
1.1488917441860464
最近鄰居中預測值為1的前5筆索引:
[128, 37, 116, 52, 125]
328    瑞昱         
237    華新         
316    誠洲         
252    友訊         
325    精英         
Name: 簡稱, dtype: object
328    113.0053
237     48.0952
316    -42.9235
252      2.0018
325     80.1532
Name: Return, dtype: float64
1.400664
2000
Number of rows in test_data: 200
confusion_matrix:
[[66 61]
 [38 35]]
classification_report
              precision    recall  f1-score   support

          -1       0.63      0.52      0.57       127
           1       0.36      0.48      0.41        73

    accuracy                           

confusion_matrix:
[[64 62]
 [35 39]]
classification_report
              precision    recall  f1-score   support

          -1       0.65      0.51      0.57       126
           1       0.39      0.53      0.45        74

    accuracy                           0.52       200
   macro avg       0.52      0.52      0.51       200
weighted avg       0.55      0.52      0.52       200

新數據的預測準確率: 0.515
1.1339807326732674
最近鄰居中預測值為1的前5筆索引:
[37, 9, 74, 78, 113]
237    華新         
209    宏電         
274    中工         
278    新纖         
313    台橡         
Name: 簡稱, dtype: object
237     48.0952
209    191.3663
274    -37.7753
278    -20.4785
313    -40.3740
Name: Return, dtype: float64
1.2816674
2000
Number of rows in test_data: 200
confusion_matrix:
[[44 83]
 [19 54]]
classification_report
              precision    recall  f1-score   support

          -1       0.70      0.35      0.46       127
           1       0.39      0.74      0.51        73

    accuracy                           0

confusion_matrix:
[[59 67]
 [22 52]]
classification_report
              precision    recall  f1-score   support

          -1       0.73      0.47      0.57       126
           1       0.44      0.70      0.54        74

    accuracy                           0.56       200
   macro avg       0.58      0.59      0.55       200
weighted avg       0.62      0.56      0.56       200

新數據的預測準確率: 0.555
1.1906427226890757
最近鄰居中預測值為1的前5筆索引:
[1, 99, 11, 52, 26]
201    華碩         
299     三商行       
211    仁寶         
252    友訊         
226     統一超       
Name: 簡稱, dtype: object
201    56.0181
299    10.7232
211    39.4893
252     2.0018
226    62.5085
Name: Return, dtype: float64
1.3414818
2000
Number of rows in test_data: 200
confusion_matrix:
[[43 84]
 [17 56]]
classification_report
              precision    recall  f1-score   support

          -1       0.72      0.34      0.46       127
           1       0.40      0.77      0.53        73

    accuracy                           0.49   